<a href="https://colab.research.google.com/github/RobynYuan/big-data-challenge/blob/main/us_Watches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-20 01:13:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.45MB/s    in 0.2s    

2022-04-20 01:13:07 (5.45 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.shell import spark
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
grocery_reveiws_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
grocery_reveiws_df.show() 


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.3
      /_/

Using Python version 3.7.13 (default, Mar 16 2022 17:37:17)
SparkSession available as 'spark'.
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|       

In [5]:
print(grocery_reveiws_df.count())

2402458


In [6]:
 grocery_reveiws_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
review_id_df=grocery_reveiws_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df_2= review_id_df.withColumn("review_date", 
                                  review_id_df["review_date"]
                                  .cast('DATE'))
review_id_df_2.printSchema()
review_id_df_2.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-08-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-08-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-08-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-08-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-08-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-08-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-08-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-08-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|   

In [8]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://amazon-reviews.corib0ca3h2a.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"root",
          "password": "motheR511037=",
          "driver":"org.postgresql.Driver"}

In [9]:
# Write DataFrame to review_id_table table in RDS

review_id_df_2.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [10]:
# -- This table will contain only unique values
# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

products_df=grocery_reveiws_df.select(['product_id','product_title'])
products_df=products_df.dropDuplicates()
products_df.printSchema()
products_df.show(5)

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0088PCJ1C|Bob's Pickle Pops...|
|B005P0U5BO|Envirokidz Organi...|
|B00K2RY8GI|Coffee Variety Sa...|
|B008233EI2|Once Again Organi...|
|B0029K0U90|Mrs. Wages ALL NA...|
+----------+--------------------+
only showing top 5 rows



In [11]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [12]:
# -- Customer table for first data set
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );
customers_df=grocery_reveiws_df.groupby(['customer_id']).agg({'customer_id':"count"})
customers_df.printSchema()
customers_df.show(5)

root
 |-- customer_id: integer (nullable = true)
 |-- count(customer_id): long (nullable = false)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   21294263|                 1|
|    8619125|                 1|
|   43626894|                 1|
|   38209321|                 4|
|   40568205|                 1|
+-----------+------------------+
only showing top 5 rows



In [13]:
customes_df=customers_df.withColumnRenamed('count(customer_id)','customer_count')
customes_df_1= customes_df.withColumn("customer_count", 
                                  customes_df["customer_count"]
                                  .cast('INTEGER'))

customes_df_1.show()
customes_df_1.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21294263|             1|
|    8619125|             1|
|   43626894|             1|
|   38209321|             4|
|   40568205|             1|
|   13480593|             1|
|   19505799|             1|
|   43150161|             2|
|   28802315|             2|
|   35329257|             1|
|   49101505|             1|
|    1117644|             1|
|   28377689|             1|
|   23493243|             1|
|   48670265|             1|
|   49041516|             3|
|   37499049|             2|
|   10920752|             1|
|    5627998|             1|
|     135423|             1|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [14]:
customes_df_1.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [18]:
# -- vine table
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );
vine_df=grocery_reveiws_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])

vine_df_2= vine_df.withColumn("star_rating", 
                                  vine_df["star_rating"]
                                  .cast('INTEGER'))
vine_df_2.printSchema()
vine_df_2.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|
| RIG9AWFOGRDVO|          2|            1|          1|   N|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|
| RB15NBVY5ELVW|          5|            2|          2|   N|
| R56358YM1ZJ7I|          5|            0|          0|   N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|
|R1

In [ ]:
vine_df_2.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)